In [ ]:
class State:
    def __init__(self, path):
        self.path = path

    def goalTest(self):
        return self.path == ["w1", "w2", "w3", "-", "-", "-", "e1", "e2", "e3"]

    def moveGen(self):
        children = []

        for i in range(9):
            current = self.path[i]

            if current.startswith("e"):
                if i+1 < 9 and self.path[i+1] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i+1] = "-", current
                    children.append(State(new_path))

                if i+2 < 9 and self.path[i+1] != "-" and self.path[i+2] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i+2] = "-", current
                    children.append(State(new_path))

            elif current.startswith("w"):
                if i-1 >= 0 and self.path[i-1] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i-1] = "-", current
                    children.append(State(new_path))

                if i-2 >= 0 and self.path[i-1] != "-" and self.path[i-2] == "-":
                    new_path = self.path.copy()
                    new_path[i], new_path[i-2] = "-", current
                    children.append(State(new_path))

        return children

    def __eq__(self, other):
        return self.path == other.path

    def __hash__(self):
        return hash(tuple(self.path))

    def __str__(self):
        return " ".join(self.path)


### Rabbit Question

- Explanation
    - Here since we have three rabbits east and three west we will consider it as
    - e1, e2, e3, w1, w2, and w3
    - Here we are using a list as  [”-", e1", "e2", "e3", "-", "w1", "w2", "w3", "-"]
- Move Gen
    - **e** rabbit can only go ++ and **w** can only go - -
    - Here the children can be as:
        1. a rabbit can go 1 step or 2 steps only if there is “-” (i.e **e** can go e+1 or e+2 and w can ****go w-1 or w-2 )
        2. a rabbit can jump only another one rabbit if there is a place after that i.e e can go e+2 directly and w can go w-2 directly
- Goal Test
    - It’s considered when all the w rabbits are left side and e rabbits are right side
    - It’s goal state is :  ["w1", "w2", "w3", "-", "-", "-", "e1", "e2", "e3" ]

### Bridge Crossing Train Question

- Explanation
    - Here we have 4 members. Boy (Amogh), Girl (Ameya), Grandpa and Grandma and each of them have some time to cross the bridge
    - We need to make all of them cross the bridge in ≤ 60 minutes, and only max 2 can go under one umbrella (They have only one umbrella)
    - Below are the timings
        
        ```
        "Amogh": 5,
        "Ameya": 10,
        "Grandma": 20,
        "Grandpa": 25
        ```
        
    - Here we are using a list ***left*** and ***right*** which is same as Men Lion Goat Cabbage Problem
    - Here we are also using a Time Left count, and the side of umbrella (left and right)
- Move Gen
    - It’s simple, if it’s boy then 5 minutes (time_left - 5), if girl then 10 minutes (time_left -10), if grandma then 25 minutes (time_left-25) and if grandpa then 20 minutes (time_left-20).
    - Here we will consider the maximum time taken as time_left - (time) so that we also have to check the **time_left ≥ 0**
- Goal Test
    - It’s considered when all of them (4) are in right list ***Right: ['Grandpa', 'Grandma', 'Amogh', 'Ameya']*** and left list is empty i.e ***Left: []*** and umbrella is at right i.e ***Umbrella: right*** and time left is ≥ 0 i.e. ***Time left: 0***

In [ ]:
class State:
    def __init__(self, left, right, umbrella, time_left):
        self.left = left          # People on left
        self.right = right        # People on right
        self.umbrella = umbrella  # Where umbrella is
        self.time_left = time_left  # Time remaining

    def goalTest(self):
        return len(self.left) == 0 and len(self.right) == 4 and self.time_left >= 0


    def moveGen(self):
        time_taken = {
            "Amogh": 5,
            "Ameya": 10,
            "Grandma": 20,
            "Grandpa": 25
        }

        children = []

        if self.umbrella == "left":
            # Try all pairs and single persons from left to right
            for i in range(len(self.left)):
                for j in range(i, len(self.left)):
                    p1 = self.left[i]
                    p2 = self.left[j]
                    t = max(time_taken[p1], time_taken[p2])
                    if self.time_left - t >= 0:
                        new_left = self.left.copy()
                        new_right = self.right.copy()

                        new_left.remove(p1)
                        if p1 != p2:
                            new_left.remove(p2)
                            new_right.append(p2)
                        new_right.append(p1)

                        child = State(new_left, new_right, "right", self.time_left - t)
                        children.append(child)

        elif self.umbrella == "right":
            # Send one person back with umbrella
            for person in self.right:
                t = time_taken[person]
                if self.time_left - t >= 0:
                    new_left = self.left.copy()
                    new_right = self.right.copy()

                    new_right.remove(person)
                    new_left.append(person)

                    child = State(new_left, new_right, "left", self.time_left - t)
                    children.append(child)

        return children
    
    def __str__(self):
         return f"Left: {self.left}, Right: {self.right}, Umbrella: {self.umbrella}, Time left: {self.time_left}"



In [ ]:
def removeSeen(children, OPEN, CLOSED):
    open_nodes = [node for node, parent in OPEN]
    closed_nodes = [node for node, parent in CLOSED]
    new_nodes = [c for c in children if c not in open_nodes and c not in closed_nodes]
    return new_nodes

def reconstructPath(node_pair, CLOSED):
    path = []
    parent_map = {}
    for node, parent in CLOSED:
        parent_map[node] = parent 
    
    node, parent = node_pair
    path.append(node)
    while parent is not None:
        path.append(parent)
        parent = parent_map[parent]
    
    return path
    

def bfs(start):
    OPEN = [(start, None)]
    CLOSED  = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            
            for p in path:
                print("->", p)
           
            return 
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(c, N) for c in new_nodes]
            OPEN = OPEN + new_pairs
    
    return []


def dfs(start):
    OPEN = [(start, None)]
    CLOSED  = []
    while OPEN:
        node_pair = OPEN.pop(0)
        N, parent = node_pair
        
        if N.goalTest():
            print("Goal found")
            path = reconstructPath(node_pair, CLOSED)
            path.reverse()
            
            for p in path:
                print("->", p)
           
            return 
        else:
            CLOSED.append(node_pair)
            children = N.moveGen()
            new_nodes = removeSeen(children, OPEN, CLOSED)
            new_pairs = [(c, N) for c in new_nodes]
            OPEN = new_pairs + OPEN
    
    return []


In [ ]:
initial = ["-","e1", "e2", "e3", "-", "w1", "w2", "w3", "-"]
start_state = State(initial)
print("BFS")
bfs(start_state)
print("DFS")
dfs(start_state)

In [ ]:
start_state = State(["Amogh", "Ameya", "Grandma", "Grandpa"], [], "left", 60)
print("BFS")
bfs(start_state)
print("DFS")
dfs(start_state)